In [1]:
import caffe
import numpy as np
import os
caffe.set_mode_gpu()


In [2]:
test=file('/home/zzq/kaggle/amazon/test_list','a+')
for i in range(40669):
    test.write('test_'+str(i)+'.jpg'+'\n')
test.close()


In [6]:
labels=open('/home/zzq/kaggle/amazon/12_class').readlines()
label=[]
for i in range(len(labels)):
    lin=labels[i].strip('\n')
    label.append(lin)

In [13]:
list_index=[]
if os.path.isfile('/home/zzq/result/amazon/result.csv'):
    os.remove('/home/zzq/result/amazon/result.csv')
net=caffe.Net('/home/zzq/caffe/models/bvlc_googlenet/deploy.prototxt',
             '/home/zzq/caffe/models/bvlc_googlenet/bvlc_googlenet_iter_10000.caffemodel',
             caffe.TEST)
out=file('/home/zzq/result/amazon/result.csv','a+')
out.write('image_name,tags'+'\n')
out.close()
root='/home/zzq/kaggle/amazon/test-jpg/'
images=open('/home/zzq/kaggle/amazon/test_list').readlines()
for i in range(len(images)):
    lin=images[i].strip('\n')
    img=root+lin
    transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})  #设定图片的shape格式(1,3,28,28)
    transformer.set_transpose('data', (2,0,1))    #改变维度的顺序，由原始图片(28,28,3)变为(3,28,28)
    #transformer.set_mean('data', np.load(mean_file).mean(1).mean(1))    #减去均值，前面训练模型时没有减均值，这儿就不用
    transformer.set_raw_scale('data', 255)    # 缩放到【0，255】之间
    transformer.set_channel_swap('data', (2,1,0))   #交换通道，将图片由RGB变为BGR
    im=caffe.io.load_image(img) 
    #net.blobs['data'].data[...] = transformer.preprocess('data',im) 
    #print lines[i]
    #net.blobs['data']=im
    net.blobs['data'].data[...] = transformer.preprocess('data',im) 
    out=net.forward()
    prob=net.blobs['prob'].data[0].flatten()
#     print type(prob)
    index=np.argmax(prob)
    list_index.append(index)
    out=file('/home/zzq/result/amazon/result.csv','a+')
    out.write('test_'+str(i)+','+label[index]+'\n')
    out.close()

In [18]:
label_replace=["agriculture clear",
              "artisinal_mine clear",
              "bare_ground clear primary",
              "blooming clear primary",
              "blow_down clear primary",
              "clear primary",
              "cloudy",
              "conventional_mine partly_cloudy",
              "cultivation partly_cloudy primary",
              "habitation partly_cloudy primary",
              "haze primary",
              "partly_cloudy primary"]
print len(label_replace)

12


## replace的效果不好，只用12类的效果并不是很好

In [19]:
if os.path.isfile('/home/zzq/result/amazon/result_replace.csv'):
    os.remove('/home/zzq/result/amazon/result_replace.csv')
out=file('/home/zzq/result/amazon/result_replace.csv','a+')
out.write('image_name,tags'+'\n')
out.close()
for i in range(len(images)):
    out=file('/home/zzq/result/amazon/result_replace.csv','a+')
    out.write('test_'+str(i)+','+label_replace[list_index[i]]+'\n')
    out.close()
    

## 这里是在使用458类来作为分类数据

In [22]:
if os.path.isfile('/home/zzq/result/amazon/result_458.csv'):
    os.remove('/home/zzq/result/amazon/result_458.csv')
out=file('/home/zzq/result/amazon/result_458.csv','a+')
out.write('image_name,tags'+'\n')
out.close()
net=caffe.Net('/home/zzq/caffe/models/bvlc_googlenet/deploy_458.prototxt',
             '/home/zzq/caffe/models/bvlc_googlenet/bvlc_googlenet_iter_400.caffemodel',
             caffe.TEST)
images=open('/home/zzq/kaggle/amazon/test_list').readlines()
root='/home/zzq/kaggle/amazon/test-jpg/'
label=[]
labels=open('/home/zzq/kaggle/amazon/458_class').readlines()
for i in range(len(labels)):
    la=labels[i].strip('\n')
    label.append(la)
for i in range(len(images)):
    lin=images[i].strip('\n')
    img=root+lin
    transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})  #设定图片的shape格式(1,3,28,28)
    transformer.set_transpose('data', (2,0,1))    #改变维度的顺序，由原始图片(28,28,3)变为(3,28,28)
    #transformer.set_mean('data', np.load(mean_file).mean(1).mean(1))    #减去均值，前面训练模型时没有减均值，这儿就不用
    transformer.set_raw_scale('data', 255)    # 缩放到【0，255】之间
    transformer.set_channel_swap('data', (2,1,0))   #交换通道，将图片由RGB变为BGR
    im=caffe.io.load_image(img) 
    #net.blobs['data'].data[...] = transformer.preprocess('data',im) 
    #print lines[i]
    #net.blobs['data']=im
    net.blobs['data'].data[...] = transformer.preprocess('data',im) 
    out=net.forward()
    prob=net.blobs['prob'].data[0].flatten()
#     print type(prob)
    index=np.argmax(prob)
    list_index.append(index)
    out=file('/home/zzq/result/amazon/result_458.csv','a+')
    out.write('test_'+str(i)+','+label[index]+'\n')
    out.close()

KeyboardInterrupt: 